In [55]:
import json
import pandas as pd

with open('../Datasets/train.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [56]:
train_data = pd.DataFrame(json_data)
train_data = train_data.drop(['tags', 'plylst_title', 'updt_date'], axis=1)
train_data.head()

,id,songs,like_cnt
0,61281,"[525514, 129701, 383374, 562083, 297861, 13954...",71
1,10532,"[432406, 675945, 497066, 120377, 389529, 24427...",1
2,76951,"[83116, 276692, 166267, 186301, 354465, 256598...",17
3,147456,"[394031, 195524, 540149, 287984, 440773, 10033...",33
4,27616,"[159327, 553610, 5130, 645103, 294435, 100657,...",9


In [57]:
train_data.shape

(115071, 3)

In [58]:
train_data = train_data.explode('songs')
train_data.head()

,id,songs,like_cnt
0,61281,525514,71
0,61281,129701,71
0,61281,383374,71
0,61281,562083,71
0,61281,297861,71


In [59]:
train_data.shape

(5285871, 3)

In [60]:
train_data.rename(columns={'songs':'song_id'}, inplace=True)
train_data.head()

,id,song_id,like_cnt
0,61281,525514,71
0,61281,129701,71
0,61281,383374,71
0,61281,562083,71
0,61281,297861,71


In [61]:
with open('../Datasets/song_meta.json', 'r', encoding='utf-8') as f:
    json_data = json.load(f)

In [62]:
song_all = pd.DataFrame(json_data)
song_all = song_all.drop(['issue_date', 'album_name', 'song_name', 'artist_name_basket', 'song_gn_gnr_basket'], axis=1)
song_all.head()

,song_gn_dtl_gnr_basket,album_id,artist_id_basket,id
0,[GN0901],2255639,[2727],0
1,"[GN1601, GN1606]",376431,[29966],1
2,[GN0901],4698747,[3361],2
3,"[GN1102, GN1101]",2644882,[838543],3
4,"[GN1802, GN1801]",2008470,[560160],4


In [63]:
song_all.shape

(707989, 4)

In [64]:
song_all.rename(columns={'song_gn_dtl_gnr_basket':'genre', 'id':'song_id'}, inplace=True)
song_all.head()

,genre,album_id,artist_id_basket,song_id
0,[GN0901],2255639,[2727],0
1,"[GN1601, GN1606]",376431,[29966],1
2,[GN0901],4698747,[3361],2
3,"[GN1102, GN1101]",2644882,[838543],3
4,"[GN1802, GN1801]",2008470,[560160],4


In [65]:
genre_list = []

for l in song_all['genre']:
    temp = dict()
    g = 0
    mx = 0
    
    for i in l:
        i = i[2:]
        n = int(i) // 100
        
        if n in temp:
            temp[n] += 1
        else:
            temp[n] = 1
    
    for i in temp:
        if mx < temp[i]:
            g = i
            mx = temp[i]
        elif mx == temp[i] and g > i:
            g = i
        
    genre_list.append(g)

song_all['genre'] = genre_list
song_all.head()

,genre,album_id,artist_id_basket,song_id
0,9,2255639,[2727],0
1,16,376431,[29966],1
2,9,4698747,[3361],2
3,11,2644882,[838543],3
4,18,2008470,[560160],4


In [70]:
merge = pd.merge(train_data, song_all)
merge.head(30)

,id,song_id,like_cnt,genre,album_id,artist_id_basket
0,61281,525514,71,14,2200223,[734201]
1,122798,525514,285,14,2200223,[734201]
2,76836,525514,8,14,2200223,[734201]
3,117966,525514,902,14,2200223,[734201]
4,19623,525514,1,14,2200223,[734201]
5,10853,525514,10,14,2200223,[734201]
6,94159,525514,132,14,2200223,[734201]
7,61281,129701,71,9,2201802,[536907]
8,122798,129701,285,9,2201802,[536907]
9,145788,129701,24,9,2201802,[536907]


In [71]:
train_target = merge['id']
train_src = merge.drop(['id', 'artist_id_basket'], axis=1)

train_src.head()

,song_id,like_cnt,genre,album_id
0,525514,71,14,2200223
1,525514,285,14,2200223
2,525514,8,14,2200223
3,525514,902,14,2200223
4,525514,1,14,2200223


In [72]:
train_target.head()

0     61281
1    122798
2     76836
3    117966
4     19623
Name: id, dtype: int64

In [77]:
from sklearn.neighbors import KNeighborsClassifier 
kn = KNeighborsClassifier(n_neighbors=1)

kn.fit(train_src, train_target)
print(kn.score(train_src, train_target))

0.6098291085802132
